<script src="https://cdn.mathjax.org/mathjax/latest/MathJax.js?config=TeX-AMS_HTML"></script>

In [ ]:
!git clone https://github.com/roeeSch/data_exercise.git

### Import necessary packages:

In [ ]:
import numpy as np
import pandas as pd
import skimage.measure

from matplotlib import pyplot as plt

### Import local packages

In [ ]:
all_data = True
analysis = False

try:
    # for colab
    from data_exercise.utils import ls2pc, ls2ft, calc_lin_char
    workdir = 'data_exercise'
except ModuleNotFoundError as e:
    # for local
    from utils import ls2pc, ls2ft, calc_lin_char
    workdir = '.'

### Load recording from csv file

In [ ]:
# Load csv:
file_name, lookingUp = workdir + '/rec_08c.csv', False
table = pd.read_table(file_name, delimiter=',')

table.iloc[:3]

**Columns are:**
1. tick is the time column (in milliseconds)
2. stateEstimate.x and stateEstimate.y are the (x,y) coordinates of spot in meters.
3. stateEstimate.yaw is spots orientation in degrees.
4. The fields {mr18.m0, mr18.m1, ..., mr18.m15} are the range measurements in millimeters. Ranges above 4000 are measurement errors.


#### Convert recorded data into desired physical units:

In [ ]:

time = table['tick'].to_numpy()
time -= time[0]
Ts = np.median(np.diff(time, 1))
print(f'original Ts={1000.0*Ts} [ms], time is [{time[0]},{time[-1]}]')

ranges = table[[f'mr18.m{i}' for i in range(16)]].to_numpy(dtype=float)*0.001
x = table['stateEstimate.x'].to_numpy()
y = table['stateEstimate.y'].to_numpy()

# unwrap angle:
t = np.unwrap(np.deg2rad(table['stateEstimate.yaw'].to_numpy()))
print(f'ranges shape before reduction: {ranges.shape}')

#### Down-sample
If needed (computations are faster with less data), downsample the recording

In [ ]:

# down-sample:
D = 1
if D>1:
    ranges = skimage.measure.block_reduce(ranges, (D,1), np.max)
    print(f'ranges shape after D=({D},1) reduction: {ranges.shape}')
    x = skimage.measure.block_reduce(x, (D,), np.median)
    y = skimage.measure.block_reduce(y, (D,), np.median)
    t = skimage.measure.block_reduce(t, (D,), np.median)
    time = skimage.measure.block_reduce(time, (D,), np.max)

Ts = np.median(np.diff(time, 1))
print(f'downsampled Ts={1000.0*Ts} [ms]')

#### Plot the x,y coordinates:

In [ ]:
plt.figure(figsize=(24,16), dpi=200)
plt.plot(x[::31], y[::31], '-b.', markersize=5)
plt.show()

## Laser Scan Vs Point Cloud
*ls2pc* - converts ranges to their corresponding x,y coordinates


In [ ]:
# This is a laser scan (LS) range array:
range_ = np.array([[1,1,2,3,4,5,6,7,11,7,6,5,4,3,2,1]])
# convert the LS to point cloud, using the information about the sensor orientations relative to each other:
x_origin = np.array([0])
y_origin = np.array([0])
yaw = np.array([0])

pc_x_r_, pc_y_r_, valid_inds_bool_, valid_inds_ = ls2pc(x_origin, y_origin, yaw, range_)

# plot the point cloud
plt.figure(figsize=(24,16), dpi=100)
plt.plot(pc_x_r_, pc_y_r_, 'm.', markersize=16)
plt.plot(0, 0, '-b.', markersize=5)
plt.show()

# play around with the other inputs of ls2pc: x_origin, y_origin, yaw.

## Add the laser scanner data


In [ ]:
# Convert laser scan (16 ranges) to point cloud (2x16 xy coordinates)
pc_x_r_, pc_y_r_, valid_inds_bool_, valid_inds_ = ls2pc(x, y, t, ranges)
plt.figure(figsize=(24,16), dpi=200)
plt.plot(pc_x_r_[valid_inds_bool_][::31], pc_y_r_[valid_inds_bool_][::31], 'm.', markersize=1)
plt.plot(x[::31], y[::31], '-b.', markersize=5)
plt.show()

**What you see above is the same data ploted in the README.md**

In blue is the odometry - the xy coordinates of spot.

In magenta - the walls\objects picked up by the range sensors.

In [ ]:
# convert laser scans to point cloud:
pc_x_r, pc_y_r, valid_inds_bool, valid_inds = ls2pc(x, y, t, ranges)

### Plot location with incrementally changing color:

In [ ]:
import matplotlib.cm as cm
c = cm.rainbow(np.linspace(0, 1, len(y[::31])))

plt.figure(figsize=(24,16), dpi=200)
plt.plot(pc_x_r_[valid_inds_bool_][::31], pc_y_r_[valid_inds_bool_][::31], 'm.', markersize=1)
plt.plot(x[::31], y[::31], alpha=0.2)
plt.scatter(x[::31], y[::31], color=c)
plt.show()

Note that the color of the location markers changes as a function of time (temporally close measurements closly colored)

### Task #1
1. Add 2 markers colored green and red. The green indicating the starting position and the red indicating the final.
2. Change the color of the odometry marker to visualize what percent of the 16 range measurements are above 2 meters. Use the color gradient 0%-black, 100%-blue.
    
  If $X_{x, y}^i$ is the location of spot at time $t_i$, and $\{r_n^i\}_{n=1}^{16}$ are the range measurements at time $t_i$, then the marker of $X_{x, y}^i$ will be colored as follows:

  $$f(\overline{r}~^i)=f(r_1^i,\ r_2^i,\ ...,\ r_{16}^i)=\sum_{n=1}^{16}\frac{1}{16}(r_n^i>2.0)$$
    
  $$Color(X_{x, y}^i)=\{R^i, G^i, B^i\} = \{0,\ 0,\ 255\cdot f(r_1^i,\ r_2^i,\ ...,\ r_{16}^i)\}$$

In [ ]:
# Enter your code here

### Task #2:
In your own words write a short explanation about what this feature ($f$) represents. If spot is in a location that is blue rather than black, what does this say about the surrounding?

### Task #3
What would you change in the above feature function $f$ so that it would be smoother and less discrete?

$|f^*(\overline{r}~^i)-f^*(\overline{r}~^{i-1})|~~<~~|f(\overline{r}~^i)-f(\overline{r}~^{i-1})|$

Add a plot with your suggestion and add a measure of smoothness that quanitfies how much smoother is your improved $f^*$ relative to $f$.

**Hint:** take a look at the **get_sigmoid** function in `utils.py`

In [ ]:
# Enter your code here

### Summary:

In this preliminary notebook we learned:
1. What the data looks like (laser scan and odometry)
2. We learned how to display the data with matplotlib
3. Finally, in the tasks, we built a single feature that indicates a type of surrounding based on a laser scan.

Next, we will add more feature functions. These will be the bassis for bulding the required classifier mentioned in the *README.md*.

In [ ]:
try:
    from data_exercise.utils import get_features_from_ranges
except ModuleNotFoundError as e:
    from utils import get_features_from_ranges

featureMat = get_features_from_ranges(ranges)

print('ranges shape = {}'.format(ranges.shape))
print('featureMat shape = {}'.format(featureMat.shape))


## Lets use these features to build a classifier:


The BD engineers want to create a classifier. A classifier that takes as inputs 16 range measurements and decides which of the 3 classes these 16 measurements belong to. The engineers do not care what the meaning of these classes are, as long as the following criterion's are met:

1. Close laser scan readings are classified to the same class with a high probability.

   **Note:** This also means that cyclically shifting a measurement should not influence the classifier.

   For example:
   $$
   \begin{aligned}
   Class(r^0_i,r^1_i,..., r^{15}_i)==Class(r^1_i,..., r^{15}_i,r^0_i)
   \end{aligned}
   $$

2. The occurrences of each class should distribute as even as possible ($1/3, 1/3, 1/3$) over a recording. 

3. The engineers do not want to manually label the data.



In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

scaler = StandardScaler()
pca_2comp = PCA(n_components=7)
kmeans_pca = KMeans(n_clusters=3, init='k-means++', random_state=42)



#### Task 4: Implement the calculation of `labeled_ranges`:

Read about these 3 functions (KMeans, StandardScaler and PCA).

Use `scaler.fit_transform`, `pca_2comp.fit_transform` and `kmeans_pca.fit_predict` to build a classifier.

In [ ]:
# Enter Your  code here 
# Hint: this can be implemented in 3 function calls
#       e.g. labeled_ranges = f1(f2(f3(featureMat)))

# labeled_ranges = ?

# What is the reason for all these stages?

In [ ]:
print(f'shape labeled_ranges = {labeled_ranges.shape}')
print(f'labeled_ranges[:6]={labeled_ranges[:6]}')

### Plot results
note: should look the same as README.md

In [ ]:
plt.figure(figsize=(24,16), dpi=200)
plt.plot(pc_x_r[valid_inds_bool][::31], pc_y_r[valid_inds_bool][::31], 'm.', markersize=1)
plt.plot(x[::31], y[::31], alpha=0.2)
plt.scatter(x[::31], y[::31], c=labeled_ranges[::31])
plt.show()

### (1) Check if close points are similarly classified:

In [ ]:
# Convert to arrays:
X = np.vstack((x, y)).T
Y = labeled_ranges
Y = np.expand_dims(Y, axis=1)

# Suggest a metric:
from sklearn.neighbors import KNeighborsRegressor
knn_model = KNeighborsRegressor(n_neighbors=3)
knn_model.fit(X, Y)

# Enter code here
# err = 
print('Probability error of predicting class based on neighbours = {}'.format(np.sum(err)/len(labeled_ranges)))

### (2) Check uniformness of class distribution:

In [ ]:
L = len(labeled_ranges)
p = []
for i in np.unique(labeled_ranges):
    p.append(np.sum(labeled_ranges==i)/L)

print('P({})={}'.format([0,1,2], [np.round(p_,2)  for p_ in p]))

from scipy.spatial.distance import jensenshannon
p_uniform = [1/3, 1/3, 1/3]
d = jensenshannon(p_uniform, p)
print('\njensenshannon distance from uniform distribution = {}'.format(d))